In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import datetime

In [2]:
# Spark session
from distribution_cost.configuration import spark_config

# Create spark session
spark_context, spark_session = spark_config.get_spark(app_name="app-distribution-cost",
                                                      executors=2, executor_cores=4, executor_mem='5g',
                                                      dynamic_allocation=True, max_executors=8)

In [3]:
# Create database uri
from distribution_cost.configuration import app
app_config = app.AppConfig()
db_uri= app_config.db_uri_jdbc
db_uri

/gpfs/user/e587246/dco00/conf/application.yml
/gpfs/user/e587246/dco00


'jdbc:oracle:thin:BRC_E587247/uOR3hLZr@//pyox2k01:1521/BRCEX_PP2'

In [4]:
df = spark_session.read.csv("/user/e587247/data/landing/arcad/GeoSiteExport_14012020_2153_31_0441_ARCAD 2020.01.14 Patrick Genolini.csv", sep=';', header=True, encoding='iso-8859-1').cache()


In [5]:
df.toPandas().head(10)

,Pays,Marque,Acteur,Identifiant du site géographique marqué,Identifiant du site géographique parent,Nom commercial,Adresse,Code postal,Ville,Code DR VN,Libellé DR VN,Code Secteur VN,Libellé Secteur VN,Code contrat,Principale/Secondaire
0,BELUX,AC,000012J,0000026290,BE00000347,ETOILE GARAGE DUDELANGE,"ROUTE DU LUXEMBOURG, 1",3515,DUDELANGE,None,None,004,LANGE JEAN-LAURENT,000012J-DVN,S
1,BELUX,AC,000012J,0000026126,BE00000219,ETOILE GARAGE ESCH,ROUTE DE BELVAUX 187,4026,ESCH SUR ALZETTE,None,None,004,LANGE JEAN-LAURENT,000012J-DVN,S
2,BELUX,AC,000012J,0000025876,BE00000004,ETOILE GARAGE S.A.R.L.,RUE ROBERT STUMPER 5,L-2557,LUXEMBOURG GASPERICH,None,None,004,LANGE JEAN-LAURENT,000012J-DVN,P
3,GERMANY,AC,000014D,0000033827,DE00000048,Autohaus Werner Beier GmbH,Heinrich-Wittmann-Str. 21,76131,Karlsruhe,300,DM VT Region Süd,332,DMV 32,000014D-DVN,P
4,GERMANY,AC,000014D,0000034595,DE00000816,Autohaus Werner Beier GmbH Betriebsstätte Ett...,Mörscher Straße 9-15,76275,Ettlingen,300,DM VT Region Süd,332,DMV 32,000014D-DVN,S
5,BELUX,AC,000014L,0000025877,BE00000005,GARAGE DE LA COLLEGIALE,RUE DE L'INDUSTRIE 33 (CT SUD),1400,NIVELLES,None,None,005,EL HAYANI NORDIN,000014L-DVN,P
6,BELUX,AC,000018Y,0000026388,BE00000426,GSL MOTORS S.A.,ROUTE DE MONS 502,6140,FONTAINE L'EVEQUE,None,None,005,EL HAYANI NORDIN,000018Y-DVN,S
7,BELUX,AC,000018Y,0000025878,BE00000006,ETS DEVERGNIES,CHAUSSEE DE MONS 476,7100,HAINE SAINT PAUL,None,None,005,EL HAYANI NORDIN,000018Y-DVN,P
8,GERMANY,AC,000019Y,0000033828,DE00000049,Autohaus Bleker GmbH,Königsberger Str. 12-14,46325,Borken,200,DM VT Region Mitte,221,DMV 21,000019Y-DVN,P
9,GERMANY,AC,000019Y,0000034481,DE00000702,"Autohaus Bleker GmbH, Betriebsstätte Münster",Kölner Straße 1,48163,Münster,200,DM VT Region Mitte,221,DMV 21,000019Y-DVN,S


In [6]:
df=df.withColumnRenamed('Pays','ID_FILIALE')
df=df.withColumnRenamed('Marque','LIB_MRQ')
df=df.withColumnRenamed('Acteur','COD_PDV')
df=df.withColumnRenamed('Identifiant du site géographique marqué','ID_SITE_MARQUE')
df=df.withColumnRenamed('Identifiant du site géographique parent','ID_SITE_PARENT')
df=df.withColumnRenamed('Nom commercial','LIB_PDV')
df=df.withColumnRenamed('Adresse','ADRESSE_PDV')
df=df.withColumnRenamed('Code postal','CODE_POSTAL_PDV')
df=df.withColumnRenamed('Ville','VILLE_PDV')
df=df.withColumnRenamed('Code DR VN','CODE_DR_PDV')
df=df.withColumnRenamed('Libellé DR VN','NOM_DR_PDV')
df=df.withColumnRenamed('Code Secteur VN','CODE_SC_PDV')
df=df.withColumnRenamed('Libellé Secteur VN','NOM_SC_PDV')
df=df.withColumnRenamed('Code contrat','CODE_CONTRAT')
df=df.withColumnRenamed('Principale/Secondaire','PRISEC')

In [7]:
#df.write.jdbc(url=db_uri, table="SMKT001_ACD_TEMP", mode="overwrite")

In [8]:
df.write.mode("overwrite").partitionBy("ID_FILIALE").parquet("hdfs:///user/e587247/data/refined/arcad/")